In [1]:
import numpy as np
import re

# Importing our translations
# for example: "spa.txt" or "spa-eng/spa.txt"
data_path = "ita.txt"

# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

# Building empty lists to hold sentences
input_docs = []
target_docs = []
# Building empty vocabulary sets
input_tokens = set()
target_tokens = set()

# Adjust the number of lines so that
# preprocessing doesn't take too long for you
for line in lines[:10000]:
  # Input and target sentences are separated by tabs
  input_doc, target_doc = line.split('\t')[:2]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)

  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below
  # and append it to target_docs:
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  # Now we split up each sentence into words
  # and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    # print(token)
    # Add your code here:
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    # print(token)
    # And here:
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

# Create num_encoder_tokens and num_decoder_tokens:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):

  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    # Assign 1. for the current line, timestep, & word
    # in encoder_input_data:
    encoder_input_data[line, timestep, input_features_dict[token]] = 1.

  for timestep, token in enumerate(target_doc.split()):

    decoder_input_data[line, timestep, target_features_dict[token]] = 1.
    if timestep > 0:

      decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

# print out those value here:


In [2]:
print(list(input_features_dict.keys())[:50], reverse_target_features_dict[50])
print(len(input_tokens))

['!', '$', '&', ',', '.', '17', '18', '19', '3', '30', '300', '45', '5', '50', '7', '8', ':', '?', 'A', 'Abandon', 'Act', 'Add', 'After', 'Aim', 'All', 'Am', 'Answer', 'Any', 'Anybody', 'Anyone', 'Anything', 'Apologize', 'Are', 'Arrive', 'Ask', 'Attack', 'Avoid', 'Awesome', 'B', 'Back', 'Be', 'Beat', 'Beef', 'Beer', "Beer's", 'Begin', 'Behave', 'Birds', 'Bless', 'Blood'] Afferrami
1607


In [ ]:
from preprocessing import num_encoder_tokens, num_decoder_tokens, decoder_target_data, encoder_input_data, decoder_input_data, decoder_target_data

from tensorflow import keras
# Add Dense to the imported layers
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# UNCOMMENT THE TWO LINES BELOW IF YOU ARE GETTING ERRORS ON A MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Choose a dimensionality
latent_dim = 52

# Choose a batch size
# and a larger number of epochs:
batch_size = 50
epochs = 50

# Encoder training setup
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder training setup:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Building the training model:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print("Model summary:\n")
training_model.summary()
print("\n\n")

# Compile the model:
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# print("Training the model:\n")
# Train the model:
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

training_model.save('training_model.h5')

2025-12-07 16:24:28.305846: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-07 16:24:28.347818: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-07 16:24:28.644735: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-07 16:24:28.854269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765121069.004633   31032 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765121069.05

Model summary:



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ 3378)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ 8480)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 52),      │    713,648 │ input_layer[0][0] │
│                     │ (None, 52),       │            │                   │
│                     │ (None, 52)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │  1,774,864 │ input_layer_1[0]… │
│                     │ 52), (None, 52),  │            │ lstm[0][1],       │
│                     │ (None, 52)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │    449,440 │ lstm_1[0][0]      │
│                     │ 8480)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,937,952 (11.21 MB)

 Trainable params: 2,937,952 (11.21 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from preprocessing import input_features_dict, target_features_dict, reverse_input_features_dict, reverse_target_features_dict, max_decoder_seq_length, input_docs, target_docs, input_tokens, target_tokens
from training_model import encoder_inputs, decoder_inputs, encoder_states, decoder_lstm, decoder_dense, encoder_input_data, num_decoder_tokens, latent_dim

from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import numpy as np

training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]

encoder_model = Model(encoder_inputs, encoder_states)


decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_sequence(test_input):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(test_input)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first token of target sequence with the start token.
  target_seq[0, 0, target_features_dict['<START>']] = 1.

  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  decoded_sentence = ''

  stop_condition = False
  while not stop_condition:
    # Run the decoder model to get possible 
    # output tokens (with probabilities) & states
    output_tokens, hidden_state, cell_state = decoder_model.predict(
      [target_seq] + states_value)

    # Choose token with highest probability
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token

    # Exit condition: either hit max length
    # or find stop token.
    if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [hidden_state, cell_state]

  return decoded_sentence


# CHANGE RANGE (NUMBER OF TEST SENTENCES TO TRANSLATE) AS YOU PLEASE
for seq_index in range(100):
  test_input = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(test_input)
  print('-')
  print('Input sentence:', input_docs[seq_index])
  print('Decoded sentence:', decoded_sentence)

In [2]:
import numpy as np
import re
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model

# UNCOMMENT IF ON MAC
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

print("Loading data...")
# Importing our translations
data_path = "ita.txt"

with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

# Building empty lists to hold sentences
input_docs = []
target_docs = []
# Building empty vocabulary sets
input_tokens = set()
target_tokens = set()

print("Preprocessing sentences...")
# Use fewer sentences to start - adjust based on RAM
NUM_SAMPLES = 5000  # Start with 5000 instead of 10000

for line in lines[:NUM_SAMPLES]:
  # Input and target sentences are separated by tabs
  input_doc, target_doc = line.split('\t')[:2]
  
  # Apply fix: separate punctuation in input too
  input_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", input_doc))
  input_docs.append(input_doc)
  
  # Separate punctuation in target
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Build vocabularies - OPTIMIZED (removed if check, .add() handles duplicates)
  for token in input_doc.split():
    input_tokens.add(token)
  
  for token in target_doc.split():
    target_tokens.add(token)

print(f"Processed {len(input_docs)} sentences")
print(f"English vocabulary: {len(input_tokens)} tokens")
print(f"Italian vocabulary: {len(target_tokens)} tokens")

# Sort vocabularies
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

# Use consistent tokenization
max_encoder_seq_length = max([len(doc.split()) for doc in input_docs])
max_decoder_seq_length = max([len(doc.split()) for doc in target_docs])

print(f"Max encoder sequence length: {max_encoder_seq_length}")
print(f"Max decoder sequence length: {max_decoder_seq_length}")

# Build dictionaries
input_features_dict = dict([(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict([(token, i) for i, token in enumerate(target_tokens)])
reverse_input_features_dict = dict((i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict((i, token) for token, i in target_features_dict.items())

print("Creating training matrices...")
encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("Filling matrices with one-hot encoded data...")
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
  if line % 1000 == 0:
    print(f"Processing sentence {line}/{len(input_docs)}...")
  
  # Use consistent tokenization (.split() instead of regex)
  for timestep, token in enumerate(input_doc.split()):
    if timestep < max_encoder_seq_length:
      encoder_input_data[line, timestep, input_features_dict[token]] = 1.
  
  for timestep, token in enumerate(target_doc.split()):
    if timestep < max_decoder_seq_length:
      decoder_input_data[line, timestep, target_features_dict[token]] = 1.
      if timestep > 0:
        decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

print("Data preprocessing complete!\n")

# MODEL BUILDING
print("Building model...")
latent_dim = 256
batch_size = 64
epochs = 50

# Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

# Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Training model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print("\nModel summary:")
training_model.summary()

print("\nCompiling model...")
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

print("\nTraining model...")
history = training_model.fit(
    [encoder_input_data, decoder_input_data], 
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    verbose=1
)

print("\nSaving model...")
training_model.save('training_model.h5')

# INFERENCE MODELS
print("\nBuilding inference models...")
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]

decoder_outputs, state_hidden, state_cell = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# TRANSLATION FUNCTION
def decode_sequence(test_input):
  states_value = encoder_model.predict(test_input, verbose=0)
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_features_dict['<START>']] = 1.
  
  decoded_sentence = ''
  stop_condition = False
  
  while not stop_condition:
    output_tokens, hidden_state, cell_state = decoder_model.predict(
        [target_seq] + states_value, verbose=0)
    
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token
    
    if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
    
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    states_value = [hidden_state, cell_state]
  
  return decoded_sentence.strip().replace(' <END>', '')

# TEST TRANSLATIONS
print("\nTesting translations on training data:")
for seq_index in range(min(20, len(input_docs))):  # Test first 20
  test_input = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(test_input)
  print('-')
  print('Input:', input_docs[seq_index])
  print('Decoded:', decoded_sentence)

Loading data...
Preprocessing sentences...
Processed 5000 sentences
English vocabulary: 939 tokens
Italian vocabulary: 2692 tokens
Max encoder sequence length: 6
Max decoder sequence length: 10
Creating training matrices...
Filling matrices with one-hot encoded data...
Processing sentence 0/5000...
Processing sentence 1000/5000...
Processing sentence 2000/5000...
Processing sentence 3000/5000...
Processing sentence 4000/5000...
Data preprocessing complete!

Building model...

Model summary:


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, None, 939) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ 2692)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 256),     │  1,224,704 │ input_layer_4[0]… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │  3,019,776 │ input_layer_5[0]… │
│                     │ 256), (None,      │            │ lstm_2[0][1],     │
│                     │ 256), (None,      │            │ lstm_2[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, None,      │    691,844 │ lstm_3[0][0]      │
│                     │ 2692)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,936,324 (18.83 MB)

 Trainable params: 4,936,324 (18.83 MB)

 Non-trainable params: 0 (0.00 B)


Compiling model...

Training model...
Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - accuracy: 0.1038 - loss: 2.9578 - val_accuracy: 0.1410 - val_loss: 2.2059
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - accuracy: 0.1377 - loss: 1.9159 - val_accuracy: 0.1415 - val_loss: 2.1807
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step - accuracy: 0.1523 - loss: 1.8215 - val_accuracy: 0.1106 - val_loss: 2.2046
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 87ms/step - accuracy: 0.1564 - loss: 1.7860 - val_accuracy: 0.1382 - val_loss: 2.1502
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.1611 - loss: 1.7503 - val_accuracy: 0.1431 - val_loss: 2.1048
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.1634 - loss: 1.7362 - val_accuracy: 0.1448 - val_loss: 2.0962
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.1653 - loss: 1.7059 - val_accuracy: 0.1429 - val_loss: 2.0768
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - accuracy: 0.1677 -


Saving model...

Building inference models...

Testing translations on training data:
-
Input: Hi .
Decoded: Aspetti !
-
Input: Hi .
Decoded: Aspetti !
-
Input: Run !
Decoded: ! !
-
Input: Run !
Decoded: ! !
-
Input: Run !
Decoded: ! !
-
Input: Who ?
Decoded: Chi ha ?
-
Input: Wow !
Decoded: ! !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Duck !
Decoded: Si !
-
Input: Jump !
Decoded: Si !
-
Input: Jump !
Decoded: Si !
-
Input: Jump !
Decoded: Si !
-
Input: Jump .
Decoded: Si .
-
Input: Jump .
Decoded: Si .
